<a href="https://colab.research.google.com/github/alexlinapp/python_tools_practice/blob/main/number_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms to convert PIL images to tensors
transform = transforms.ToTensor()

# Download and load MNIST training data
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Download and load MNIST test data
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [62]:
dl_train = DataLoader(mnist_train, batch_size=16)
xb, yb = next(iter(dl_train))
xb.shape, yb.shape
xb = xb.view(xb.size(0), -1)
yb = yb.view(yb.size(0), -1)

tensor([[5],
        [0],
        [4],
        [1],
        [9],
        [2],
        [1],
        [3],
        [1],
        [4],
        [3],
        [5],
        [3],
        [6],
        [1],
        [7]])

In [52]:
def init_params(size, std=1.0):
  return (torch.randn(size)*std).requires_grad_()
def mnist_loss(prediction, target):
  prediction = prediction.sigmoid()
  return torch.where(prediction, )

In [49]:
weights = init_params((xb.size(1), 1))
bias = init_params(1)
def linear1(x):
  return x@weights + bias

In [51]:
preds = linear1(xb)
preds

tensor([[13.7674],
        [ 7.0368],
        [ 8.9417],
        [20.7591],
        [19.0512],
        [24.6774],
        [-2.5220],
        [24.0587],
        [-3.2825],
        [ 8.8620],
        [12.6664],
        [ 6.8320],
        [ 6.6949],
        [13.2273],
        [-3.6416],
        [14.9295]], grad_fn=<AddBackward0>)